In [1]:
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [2]:
!pip install catboost

In [3]:
import os
import time
import numpy as np
import pandas as pd
import featuretools as ft
import catboost
from tqdm import tqdm_notebook
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.optimize
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, f1_score
from sklearn.externals import joblib
# from category_encoders import CatBoostEncoder

os.chdir("/content/Drive/My Drive/Colab Notebooks/2019玉山信用卡盜刷比賽/(DAServer_CJ)T-brain_玉山信用卡盜刷201909")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
# train_raw = pd.read_csv("train_pre_31.csv")
# X_test = pd.read_csv("test_pre_31.csv")
# train_raw = train_raw.drop_duplicates(subset=train_raw.columns[~train_raw.columns.isin([
#         'acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
#        'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'mcc',
#        'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', "loctm_min", 
#        'conam_mean_by_loctm_min', 
#        'conam_mean_by_loctm_sec', 
#        'bacno_cano_no_recent_10_conam_pos_trend',
#        'bacno_cano_no_recent_7_loctm_diff', 
#        'bacno_cano_loctm_uique_count', 
#        'bacno_cano_14_rolling_times_prime_time_fraud_cnt',
#        'bacno_cano_10_times_sleep_time_cnt',
#        'bacno_cano_10_sleep_time_conam_lower_3000_cnt',
#        'bacno_cano_10_prime_time_fraud_conam_lower_3000_cnt', 
#        'bacno_cano_no_recent_10_ecfg_sum', 'fraud_ind', "locdt",
#         'flbmk_value_counts', 'hcefg_value_counts', 'flg_3dsmk_value_counts'
#          'ecfg_value_counts', 'ovrlt_value_counts', 'insfg_value_counts'
#         "conam_mean_by_train_test_acqic", "loctm_value_counts",
#         "loctm_hour_value_counts", 'loctm_sec'])], keep='last')
# test_raw = pd.read_csv("test.csv")

# X = train_raw.drop('fraud_ind', axis=1)
# y = train_raw['fraud_ind']

# print(train_raw.shape)
# print(X_test.shape)
# print(test_raw.shape)

In [0]:
# Read data
train_raw = pd.read_csv("engineered_data/train_pre_31.csv")
X_test = pd.read_csv("engineered_data/test_pre_31.csv")

# train_raw = pd.read_csv("train_pre_13.csv")
# X_test = pd.read_csv("test_pre_13.csv")

# train_raw = pd.read_csv("train_pre_pure.csv")
# X_test = pd.read_csv("test_pre_pure.csv")

In [0]:
train_raw = train_raw.drop_duplicates(subset=train_raw.columns[~train_raw.columns.isin([
        'acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt', 'mcc',
       'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', "loctm_min", 
       'conam_mean_by_loctm_min', 
       'conam_mean_by_loctm_sec', 
       'bacno_cano_no_recent_10_conam_pos_trend',
       'bacno_cano_no_recent_7_loctm_diff', 
       'bacno_cano_loctm_uique_count', 
       'bacno_cano_14_rolling_times_prime_time_fraud_cnt',
       'bacno_cano_10_times_sleep_time_cnt',
       'bacno_cano_10_sleep_time_conam_lower_3000_cnt',
       'bacno_cano_10_prime_time_fraud_conam_lower_3000_cnt', 
       'bacno_cano_no_recent_10_ecfg_sum', 'fraud_ind', "locdt",
        'flbmk_value_counts', 'hcefg_value_counts', 'flg_3dsmk_value_counts'
         'ecfg_value_counts', 'ovrlt_value_counts', 'insfg_value_counts'
        "conam_mean_by_train_test_acqic", "loctm_value_counts",
        "loctm_hour_value_counts", 'loctm_sec'])], keep='last')

# train_raw = train_raw.drop_duplicates(subset=train_raw.columns[train_raw.columns.isin([
#                                     "acqic", "cano", 
#                                     "conam", "csmcu", "ecfg", "etymd", 
#                                     "flbmk", "flg_3dsmk" ,"hcefg", "insfg", "iterm",
#                                     "mcc", "mchno", "ovrlt" ,"scity", "stocn", "stscd", "loctm_hour"])], keep='last')

In [6]:
test_raw = pd.read_csv("raw_data/test.csv")

X = train_raw.drop('fraud_ind', axis=1)
y = train_raw['fraud_ind']

print(train_raw.shape)
print(X_test.shape)
print(test_raw.shape)

(1461618, 115)
(421665, 114)
(421665, 22)


In [7]:
all_features = pd.concat([X, X_test], axis=0)
all_features.tail()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,loctm_hour,loctm_min,loctm_sec,conam_is_0,bacno_u_cnt_by_acqic,conam_level,loctm_value_counts,loctm_hour_value_counts,csmcu_cnt_by_stocn,csmcu_cnt_by_scity,csmcu_ucnt_by_bacno,bacno_value_counts,cano_value_counts,mchno_value_counts,acqic_value_counts,mcc_value_counts,ecfg_value_counts,insfg_value_counts,contp_value_counts,etymd_value_counts,...,stscd_uique_count_by_bacno,csmcu_uique_count_by_bacno,bacno_cano_mcc_acqic_cnt_by_locdt,bacon_cano_cnt,bacno_cano_loctm_uique_count,bacno_cano_14_rolling_times_prime_time_fraud_cnt,bacno_cano_10_times_sleep_time_cnt,bacno_cano_10_sleep_time_conam_lower_3000_cnt,bacno_cano_10_prime_time_fraud_conam_lower_3000_cnt,bacno_cano_no_recent_10_conam_pos_trend,bacno_cano_no_recent_7_loctm_diff,bacno_cano_no_recent_10_ecfg_sum,bacno_cnt_by_locdt,bacno_cnt_by_locdt_mchno,bacno_cnt_by_locdt_conam,bacno_cnt_by_locdt_acqic,bacno_cnt_by_locdt_mcc,bacno_cnt_by_locdt_scity,bacno_cnt_by_locdt_stscd,bacno_cnt_by_locdt_csmcu,bacno_cnt_by_locdt_loctm_hour,bacno_cnt_each_hour,conam_cnt_each_hour,cano_cnt_each_hour,mchno_cnt_each_hour,acqic_cnt_each_hour,mcc_cnt_each_hour,scity_cnt_each_hour,stscd_cnt_each_hour,csmcu_cnt_each_hour,mchno_cnt_by_locdt_loctm_hour,acqic_cnt_by_locdt_loctm_hour,conam_cnt_by_locdt_loctm_hour,mcc_cnt_by_locdt_loctm_hour,scity_cnt_by_locdt_loctm_hour,stscd_cnt_by_locdt_loctm_hour,csmcu_cnt_by_locdt_loctm_hour,conam_is_0_cnt_by_bacno,conam_mean_by_train_test_acqic,acqic_mcc_counts
421660,5975,147326,79511,633.76,5,62,0,2,0,0,5,0,0,116,343,87282,0,5817,102,0,11,12,28,0,41913,0,32,108543,9,5,1,1,1,2099,158743,79273,1425835,1841385,1696824,468818,...,1,1,1,1,1,8.0,3.0,0,0,34.320000,5.811905,9.0,1,1,1,1,1,1,1,1,1,1,22,1,764,11365,8247,54790,107875,93247,63,98,2,97,535,907,810,0,627.570710,9005
421661,6716,14305,136493,952.84,5,62,0,5,0,0,8,0,0,107,245,48784,0,5859,102,0,8,58,39,0,74622,0,26,61609,9,2,1,1,1,1635,249498,8679,1425835,1841385,1696824,511459,...,1,1,1,1,1,4.0,0.0,0,0,169.797500,137.312500,2.0,1,1,1,1,1,1,1,1,1,1,4,1,105,4417,449,518,61187,49699,1,27,1,3,8,380,306,0,797.537255,6995
421662,5975,156543,137963,713.42,5,62,0,4,0,0,5,0,0,111,263,98326,0,5817,102,0,18,49,21,0,41913,0,35,126370,9,5,1,2,1,61,158743,138939,1425835,1841385,1696824,400339,...,1,1,1,1,1,2.0,0.0,0,0,-37.223333,121.366667,0.0,1,1,1,1,1,1,1,1,1,1,23,1,6,11281,10398,58364,125691,107666,2,110,2,97,459,950,794,0,627.570710,63643
421663,6231,156543,137964,903.94,5,62,0,5,0,0,5,0,0,114,251,17763,0,5817,102,0,14,44,34,0,20656,0,38,119420,9,5,1,2,1,322,68948,243998,1425835,1841385,1696824,511459,...,1,1,1,1,1,1.0,0.0,0,0,0.000000,0.000000,0.0,1,1,1,1,1,1,1,1,1,1,9,1,24,4946,16319,56672,118742,104569,1,43,1,162,478,1043,913,0,666.248912,10525
421664,6769,131985,98699,760.09,5,62,0,4,0,0,5,0,0,116,251,78019,0,3294,102,0,16,28,4,0,72630,0,36,126410,9,2,1,1,1,1862,318569,243998,1425835,1841385,1696824,400339,...,1,1,1,1,1,7.0,2.0,0,0,6.298000,16.561905,7.0,1,1,1,1,1,1,1,1,1,1,20,1,194,22415,21227,600,125780,110247,2,154,1,125,5,850,726,0,649.901595,167308


In [8]:
for col in X.columns:
    print(col)
    print(X[col].unique().shape, "&", all_features[col].unique().shape)

acqic
(6051,) & (6866,)
bacno
(94767,) & (162739,)
cano
(128929,) & (211975,)
conam
(70279,) & (83982,)
contp
(7,) & (7,)
csmcu
(72,) & (76,)
ecfg
(2,) & (2,)
etymd
(11,) & (11,)
flbmk
(2,) & (2,)
flg_3dsmk
(2,) & (2,)
hcefg
(9,) & (10,)
insfg
(2,) & (2,)
iterm
(9,) & (9,)
locdt
(90,) & (120,)
mcc
(434,) & (460,)
mchno
(89299,) & (102766,)
ovrlt
(2,) & (2,)
scity
(5697,) & (6646,)
stocn
(103,) & (109,)
stscd
(5,) & (5,)
loctm_hour
(24,) & (24,)
loctm_min
(60,) & (60,)
loctm_sec
(60,) & (60,)
conam_is_0
(2,) & (2,)
bacno_u_cnt_by_acqic
(259,) & (259,)
conam_level
(4,) & (4,)
loctm_value_counts
(80,) & (80,)
loctm_hour_value_counts
(24,) & (24,)
csmcu_cnt_by_stocn
(20,) & (20,)
csmcu_cnt_by_scity
(19,) & (19,)
csmcu_ucnt_by_bacno
(15,) & (15,)
bacno_value_counts
(233,) & (236,)
cano_value_counts
(191,) & (195,)
mchno_value_counts
(774,) & (774,)
acqic_value_counts
(310,) & (310,)
mcc_value_counts
(228,) & (228,)
ecfg_value_counts
(2,) & (2,)
insfg_value_counts
(2,) & (2,)
contp_value_cou

In [0]:
del all_features["locdt"]
# del all_features['flg_3dsmk_value_counts']
# del all_features['flbmk_value_counts']
# del all_features['hcefg_value_counts']
# del all_features['ecfg_value_counts']
# del all_features['ovrlt_value_counts']
# del all_features['insfg_value_counts']
# del all_features['conam_mean_by_loctm_hour']
# del all_features['conam_mean_by_loctm_min']
# del all_features['conam_mean_by_loctm_sec']
# del all_features["bacno_cano_10_prime_time_fraud_conam_lower_3000_cnt"]
# del all_features["prime_time_fraud_conam_lower_3000"]
# del all_features["hcefg"]
# del all_features["ovrlt"]
# del all_features["insfg"]
# del all_features["acqic"]
# del all_features["bacno"]
# del all_features["cano"]
# del all_features["contp"]
# del all_features["csmcu"]
# del all_features["ecfg"]
# del all_features["etymd"]
# del all_features["scity"]
# del all_features["stocn"]
# del all_features["mcc"]
# del all_features["mchno"]

# 這裡
del all_features["conam_mean_by_train_test_acqic"]

del all_features["loctm_value_counts"]
del all_features["loctm_hour_value_counts"]
del all_features['loctm_sec']


## 去掉交叉特徵 --> 沒用
# for col in ['bacno_cano','bacno_mchno','bacno_acqic','bacno_mcc','bacno_scity','bacno_stscd','bacno_csmcu','cano_mchno','cano_acqic','cano_mcc',
#  'cano_scity','cano_stscd','cano_csmcu','mchno_acqic','mchno_mcc','mchno_scity','mchno_stscd','mchno_csmcu','acqic_mcc','acqic_scity',
#  'acqic_stscd','acqic_csmcu','mcc_scity','mcc_stscd','mcc_csmcu','scity_stscd','scity_csmcu','stscd_csmcu']:
#     del all_features[col]

## 去掉不小心重複放的特徵
# del all_features["mchno_cnt_by_locdt_loctm_hour_y"]
# del all_features["acqic_cnt_by_locdt_loctm_hour_y"]
# del all_features["conam_cnt_by_locdt_loctm_hour_y"]
# del all_features["mcc_cnt_by_locdt_loctm_hour_y"]
# del all_features["scity_cnt_by_locdt_loctm_hour_y"]
# del all_features["stscd_cnt_by_locdt_loctm_hour_y"]
# del all_features["csmcu_cnt_by_locdt_loctm_hour_y"]

# repeat_cols = ["mchno_cnt_by_locdt_loctm_hour_x", "acqic_cnt_by_locdt_loctm_hour_x", "conam_cnt_by_locdt_loctm_hour_x",
#               "mcc_cnt_by_locdt_loctm_hour_x", "scity_cnt_by_locdt_loctm_hour_x", "stscd_cnt_by_locdt_loctm_hour_x",
#               "csmcu_cnt_by_locdt_loctm_hour_x"]
# for col in repeat_cols:
#     all_features.rename(columns={col: col[:-2]}, inplace=True)

# del all_features["bacno_cano_recent_no_5_loctm_acc_min_diff"]
# del all_features["bacno_cano_no_mean_loctm_acc_min_diff"]
# del all_features["bacno_cano_7_times_conam_max_by_no"]
# rolling系列先不要
# del all_features["bacno_cano_conam_rolling_3_window_mean"]
# del all_features["bacno_cano_conam_rolling_7_window_sum"]
# del all_features["bacno_cano_conam_cumsum_mean"]
# del all_features["bacno_cano_conam_rolling_14_window_max"]

# # all_features["bacno_cano_conam_rolling_3_window_mean"] = pd.cut(all_features["bacno_cano_conam_rolling_3_window_mean"], 30000, labels=False)
# # all_features["bacno_cano_conam_rolling_7_window_mean"] = pd.cut(all_features["bacno_cano_conam_rolling_7_window_mean"], 20000, labels=False)

# # all_features["bacno_cano_no_recent_7_conam_pos_trend"] = pd.cut(all_features["bacno_cano_no_recent_7_conam_pos_trend"], 20000, labels=False)
# all_features["bacno_cano_min_diff_latter"] = pd.cut(all_features["bacno_cano_min_diff_latter"], 10000, labels=False)
# all_features["bacno_cano_recent_no_5_loctm_acc_min_diff"] = pd.cut(all_features["bacno_cano_recent_no_5_loctm_acc_min_diff"], 10000, labels=False)

In [10]:
# del all_features["bacno_cano_14_rolling_times_prime_time_fraud_cnt"]
# del all_features["bacno_cano_min_diff_latter"]

X = all_features[:-len(X_test)]
X_test = all_features[-len(X_test):]
print(X.shape)
print(X_test.shape)

(1461618, 109)
(421665, 109)


## 資料處理

In [11]:
for col in X.columns:
    print(col)
    print(X[col].unique().shape)

acqic
(6051,)
bacno
(94767,)
cano
(128929,)
conam
(70279,)
contp
(7,)
csmcu
(72,)
ecfg
(2,)
etymd
(11,)
flbmk
(2,)
flg_3dsmk
(2,)
hcefg
(9,)
insfg
(2,)
iterm
(9,)
mcc
(434,)
mchno
(89299,)
ovrlt
(2,)
scity
(5697,)
stocn
(103,)
stscd
(5,)
loctm_hour
(24,)
loctm_min
(60,)
conam_is_0
(2,)
bacno_u_cnt_by_acqic
(259,)
conam_level
(4,)
csmcu_cnt_by_stocn
(20,)
csmcu_cnt_by_scity
(19,)
csmcu_ucnt_by_bacno
(15,)
bacno_value_counts
(233,)
cano_value_counts
(191,)
mchno_value_counts
(774,)
acqic_value_counts
(310,)
mcc_value_counts
(228,)
ecfg_value_counts
(2,)
insfg_value_counts
(2,)
contp_value_counts
(7,)
etymd_value_counts
(11,)
stocn_value_counts
(73,)
scity_value_counts
(304,)
stscd_value_counts
(5,)
ovrlt_value_counts
(2,)
flbmk_value_counts
(2,)
hcefg_value_counts
(9,)
csmcu_value_counts
(56,)
flg_3dsmk_value_counts
(2,)
prime_time_normal
(2,)
prime_time_fraud
(2,)
sleep_time
(2,)
sleep_time_conam_lower_3000
(2,)
prime_time_fraud_conam_lower_3000
(2,)
conam_mean_by_bacno
(87764,)
conam_m

In [0]:
cat_colnames = ["bacno", "cano", "mchno", "acqic", "mcc", "ecfg", "insfg", "contp", 'etymd',
                "stocn", "scity", "stscd", "ovrlt", "flbmk", "hcefg", "csmcu", "flg_3dsmk"] # + high_cacol

categorical_feature_idx = [i for i, col in enumerate(X.columns) if col in cat_colnames]

In [13]:
model_type = 'cat'
for col in X.columns[categorical_feature_idx]:
    # for catboost
    if model_type == 'cat':
        X[col] = X[col].astype('str')
        X_test[col] = X_test[col].astype('str')
        print(col)
        print(X[col].dtype)
    else:
        X[col] = X[col].astype('category')
        X_test[col] = X_test[col].astype('category')
        print(col)
        print(X[col].dtype)

acqic
object
bacno
object
cano
object
contp
object
csmcu
object
ecfg
object
etymd
object
flbmk
object
flg_3dsmk
object
hcefg
object
insfg
object
mcc
object
mchno
object
ovrlt
object
scity
object
stocn
object
stscd
object


In [14]:
# shuffle一下
# perm = np.random.permutation(len(X))
# X = X.iloc[perm].reset_index(drop=True) 
# y = y.iloc[perm].reset_index(drop=True)

# Seed = 111
sss = StratifiedKFold(n_splits=4, random_state=111, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

Train: [ 361384  361385  361386 ... 1461615 1461616 1461617] Test: [      0       1       2 ... 1449552 1449553 1449554]
Train: [      0       1       2 ... 1461615 1461616 1461617] Test: [ 361384  361385  361386 ... 1453573 1453574 1453575]
Train: [      0       1       2 ... 1461615 1461616 1461617] Test: [ 722768  722769  722770 ... 1457594 1457595 1457596]
Train: [      0       1       2 ... 1457594 1457595 1457596] Test: [1084151 1084152 1084153 ... 1461615 1461616 1461617]
(1096214, 109)
(1096214,)
(365404, 109)
(365404,)


## Catboost

In [0]:
# train不起來可能是因為cat index搞錯或有Na/ Cat對training set size很敏感?? -> 不，是去掉village好很多
# Catboost

model_cat = catboost.CatBoostClassifier(iterations=500,
                              learning_rate=0.005,
                              depth=6,
                              eval_metric='AUC',
                              scale_pos_weight=2,
                              # random_seed=11,
                              # l2_leaf_reg=4,
                              # one_hot_max_size=100,
                              # rsm=0.5,
                              metric_period=100,
                              task_type="GPU",
                              # devices='0:1',
                              # plot=True,
                              # subsample=0.5,
                              early_stopping_rounds=100)


In [0]:
model_cat.fit(X_train, y_train, eval_set=(X_val, y_val), 
              cat_features=categorical_feature_idx, 
              # use_best_model=True,
              verbose=True)

0:	learn: 0.9758967	test: 0.9804895	best: 0.9804895 (0)	total: 117ms	remaining: 58.3s
100:	learn: 0.9838165	test: 0.9867432	best: 0.9867542 (99)	total: 12.5s	remaining: 49.5s
200:	learn: 0.9854959	test: 0.9882111	best: 0.9882111 (200)	total: 24.9s	remaining: 37s
300:	learn: 0.9875949	test: 0.9903440	best: 0.9903440 (300)	total: 37.7s	remaining: 24.9s
400:	learn: 0.9886565	test: 0.9913146	best: 0.9913146 (400)	total: 50.8s	remaining: 12.5s
499:	learn: 0.9893420	test: 0.9919177	best: 0.9919177 (499)	total: 1m 3s	remaining: 0us
bestTest = 0.9919176996
bestIteration = 499


In [0]:
model_cat.save_model("model_saved/pre_31_final_cat_v1")

In [16]:
model_cat.load_model("model_saved/pre_31_final_cat_v1")

CatBoostError: ignored

In [1]:
model_cat

NameError: ignored